In [61]:
# Utils packages
import ast
from pathlib import Path, PurePath
from typing import Union

# Data collector packages
from hentai import Utils, Hentai, Option

# Analysis packages
import pandas as pd
from PIL import Image
from tqdm import tqdm

# Compute packages
import numpy as np
import tensorflow as tf

## Data collection

We will use the information and media collected on the nhentai.net website. Recording each entry in a 'metadata.csv' file that aggregates information about the hentai, including links to download pages.

In [62]:
N_SAMPLE = 1 # Number of sample to refetch from hentai.
DATA_PATH = "data" # Directory to drop collected data.

In [63]:
def get_random_hentai(_: int):
    return pd.Series(Utils.get_random_hentai().dictionary(Option.all()))

In [64]:
# Query the nhentai.net API to get N number of random hentai
collected_hentais_df = pd.DataFrame(range(N_SAMPLE))
collected_hentais_df = collected_hentais_df.apply(get_random_hentai, axis=1)
collected_hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages
0,355066,BETWEEN US Ch. 1-36,,https://nhentai.net/g/355066,https://nhentai.net/api/gallery/355066,1888032,1618291299,1796,"[big breasts, full censorship, story arc, mast...",[],[],[],"[english, translated]",[],[manga],https://t.nhentai.net/galleries/1888032/cover.jpg,https://t.nhentai.net/galleries/1888032/thumb.jpg,[https://i.nhentai.net/galleries/1888032/1.jpg...,287


## Read dataset file

In [65]:
# The metadata NDJSON file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "raw" / "metadata.ndjson"

In [66]:
hentais_df = pd.read_json(
    metadata_path,
    orient="records",
    lines=True
)
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages
0,338065,NI+CORA de SHI+CORA Night,,https://nhentai.net/g/338065,https://nhentai.net/api/gallery/338065,1788827,1606885806,314,"[sole male, dark skin, ffm threesome, group, a...",[charan poran],[tokyo 7th sisters],"[alessandra susu, musubi tendouji]",[japanese],[nekono matatabi],[doujinshi],https://t.nhentai.net/galleries/1788827/cover.jpg,https://t.nhentai.net/galleries/1788827/thumb.jpg,[https://i.nhentai.net/galleries/1788827/1.jpg...,14
1,352167,Boku wa Chiisana Succubus no Shimobe - I am a ...,None,https://nhentai.net/g/352167,https://nhentai.net/api/gallery/352167,1872914,1616460447,20119,"[wings, garter belt, group, uncensored, story ...",[],[],[],"[english, translated]",[tamano kedama],[manga],https://t.nhentai.net/galleries/1872914/cover.jpg,https://t.nhentai.net/galleries/1872914/thumb.jpg,[https://i.nhentai.net/galleries/1872914/1.jpg...,195
2,212548,Syoku,None,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],[occultic nine],[ririka nishizono],"[english, translated]",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33
3,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,None,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],[],[],[japanese],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166
4,75495,SHIRONA HAZARD,None,https://nhentai.net/g/75495,https://nhentai.net/api/gallery/75495,490119,1403996013,191,[big breasts],[draw go],[pokemon],[cynthia],[japanese],[souichi],[doujinshi],https://t.nhentai.net/galleries/490119/cover.jpg,https://t.nhentai.net/galleries/490119/thumb.jpg,"[https://i.nhentai.net/galleries/490119/1.jpg,...",23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,19145,Fuuka Meiga za Nihon date: Roommate no Hime go...,None,https://nhentai.net/g/19145,https://nhentai.net/api/gallery/19145,68021,1403984982,17,"[lolicon, yuri]",[bolze.],[mai-otome],"[shizuru fujino, erstin ho, nina wang, yohko h...",[japanese],[rit.],[doujinshi],https://t.nhentai.net/galleries/68021/cover.jpg,https://t.nhentai.net/galleries/68021/thumb.jpg,"[https://i.nhentai.net/galleries/68021/1.jpg, ...",18
3336,243825,Count2.4 CGF Soushuuhen,,https://nhentai.net/g/243825,https://nhentai.net/api/gallery/243825,1274983,1534955886,1167,"[bondage, anal, gag, nakadashi, stockings]",[count2.4],[the idolmaster],"[koume shirasaka, mayu sakuma, mika jougasaki,...",[japanese],[nishi],[doujinshi],https://t.nhentai.net/galleries/1274983/cover.jpg,https://t.nhentai.net/galleries/1274983/thumb.jpg,[https://i.nhentai.net/galleries/1274983/1.jpg...,140
3337,230693,ふたりきり,,https://nhentai.net/g/230693,https://nhentai.net/api/gallery/230693,1213137,1524110819,70,"[anal, dark skin, males only, shotacon, yaoi, ...",[],[dragon ball z],"[son goku, turles]",[japanese],[],[doujinshi],https://t.nhentai.net/galleries/1213137/cover.png,https://t.nhentai.net/galleries/1213137/thumb.png,[https://i.nhentai.net/galleries/1213137/1.png...,15
3338,115897,Otoko no Tatakai 18,,https://nhentai.net/g/115897,https://nhentai.net/api/gallery/115897,736813,1410126146,49,"[fundoshi, humiliation, full censorship, femdo...",[ito ichizo shouten],[],[],[japanese],[ito ichizo],[doujinshi],https://t.nhentai.net/galleries/736813/cover.jpg,https://t.nhentai.net/galleries/736813/thumb.jpg,"[https://i.nhentai.net/galleries/736813/1.jpg,...",17


## Save collected dataset

In [67]:
hentais_df = pd.concat([collected_hentais_df, hentais_df], ignore_index=True)
hentais_df = hentais_df.drop_duplicates("id")
hentais_df.to_json(
    metadata_path,
    orient="records",
    lines=True
)

## Download images

In [68]:
def download_hentai(hentai: pd.Series):
    # A hentai will be stored based on the gallery ID instead of the medai ID
    hentai_path = Path(DATA_PATH) / "raw" / str(hentai.id)
    # Since downloading a hentai is an expensive operation, we only
    # download when the hentai ID directory does not exist
    if not hentai_path.is_dir():
        try:
            hentai = Hentai(hentai.id)
            hentai.download(folder=hentai_path, progressbar=True)
            return True
        except Exception:
            pass
    return False

In [69]:
downloaded_hentais_df = hentais_df.apply(download_hentai, axis=1)

Download #355066: 100%|○○○○○○○○○○| 287/287 [02:19<00:00,  2.06page/s]


In [70]:
print("Number of downloaded hentais :", downloaded_hentais_df.sum())

Number of downloaded hentais : 1


## Remove corrupted images

In [71]:
def hentai_filenames(hentai: pd.Series):
    return [
        str(PurePath(str(hentai["id"])) / Path(url).name)
        for url in hentai["image_urls"]
    ]

In [72]:
hentais_df = hentais_df.assign(
    filenames=hentais_df.apply(hentai_filenames, axis=1)
)
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages,filenames
0,355066,BETWEEN US Ch. 1-36,,https://nhentai.net/g/355066,https://nhentai.net/api/gallery/355066,1888032,1618291299,1796,"[big breasts, full censorship, story arc, mast...",[],[],[],"[english, translated]",[],[manga],https://t.nhentai.net/galleries/1888032/cover.jpg,https://t.nhentai.net/galleries/1888032/thumb.jpg,[https://i.nhentai.net/galleries/1888032/1.jpg...,287,"[355066\1.jpg, 355066\2.jpg, 355066\3.jpg, 355..."
1,338065,NI+CORA de SHI+CORA Night,,https://nhentai.net/g/338065,https://nhentai.net/api/gallery/338065,1788827,1606885806,314,"[sole male, dark skin, ffm threesome, group, a...",[charan poran],[tokyo 7th sisters],"[alessandra susu, musubi tendouji]",[japanese],[nekono matatabi],[doujinshi],https://t.nhentai.net/galleries/1788827/cover.jpg,https://t.nhentai.net/galleries/1788827/thumb.jpg,[https://i.nhentai.net/galleries/1788827/1.jpg...,14,"[338065\1.jpg, 338065\2.jpg, 338065\3.jpg, 338..."
2,352167,Boku wa Chiisana Succubus no Shimobe - I am a ...,None,https://nhentai.net/g/352167,https://nhentai.net/api/gallery/352167,1872914,1616460447,20119,"[wings, garter belt, group, uncensored, story ...",[],[],[],"[english, translated]",[tamano kedama],[manga],https://t.nhentai.net/galleries/1872914/cover.jpg,https://t.nhentai.net/galleries/1872914/thumb.jpg,[https://i.nhentai.net/galleries/1872914/1.jpg...,195,"[352167\1.jpg, 352167\2.jpg, 352167\3.jpg, 352..."
3,212548,Syoku,None,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],[occultic nine],[ririka nishizono],"[english, translated]",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33,"[212548\1.jpg, 212548\2.jpg, 212548\3.jpg, 212..."
4,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,None,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],[],[],[japanese],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166,"[57170\1.jpg, 57170\2.jpg, 57170\3.jpg, 57170\..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3336,19145,Fuuka Meiga za Nihon date: Roommate no Hime go...,None,https://nhentai.net/g/19145,https://nhentai.net/api/gallery/19145,68021,1403984982,17,"[lolicon, yuri]",[bolze.],[mai-otome],"[shizuru fujino, erstin ho, nina wang, yohko h...",[japanese],[rit.],[doujinshi],https://t.nhentai.net/galleries/68021/cover.jpg,https://t.nhentai.net/galleries/68021/thumb.jpg,"[https://i.nhentai.net/galleries/68021/1.jpg, ...",18,"[19145\1.jpg, 19145\2.jpg, 19145\3.jpg, 19145\..."
3337,243825,Count2.4 CGF Soushuuhen,,https://nhentai.net/g/243825,https://nhentai.net/api/gallery/243825,1274983,1534955886,1167,"[bondage, anal, gag, nakadashi, stockings]",[count2.4],[the idolmaster],"[koume shirasaka, mayu sakuma, mika jougasaki,...",[japanese],[nishi],[doujinshi],https://t.nhentai.net/galleries/1274983/cover.jpg,https://t.nhentai.net/galleries/1274983/thumb.jpg,[https://i.nhentai.net/galleries/1274983/1.jpg...,140,"[243825\1.jpg, 243825\2.jpg, 243825\3.jpg, 243..."
3338,230693,ふたりきり,,https://nhentai.net/g/230693,https://nhentai.net/api/gallery/230693,1213137,1524110819,70,"[anal, dark skin, males only, shotacon, yaoi, ...",[],[dragon ball z],"[son goku, turles]",[japanese],[],[doujinshi],https://t.nhentai.net/galleries/1213137/cover.png,https://t.nhentai.net/galleries/1213137/thumb.png,[https://i.nhentai.net/galleries/1213137/1.png...,15,"[230693\1.png, 230693\2.png, 230693\3.png, 230..."
3339,115897,Otoko no Tatakai 18,,https://nhentai.net/g/115897,ht

We will filter every hentais that contain at least one corrupted images, since pages have order dependencies.

In [73]:
def preprocess_image(filename: str):
    image_path = Path(filename)
    image_preproc_dir = Path(DATA_PATH) / "preprocessed" / image_path.parent
    image_preproc_path = image_preproc_dir / Path(image_path.stem + ".jpg")
    # Preprocess only those images that do not exist in
    # the destination dataset for performance reason
    if not image_preproc_path.is_file():
        try:
            image = Image.open(Path(DATA_PATH) / "raw" / image_path)
            # Normalize images to be JPEG compatible
            image = image.convert("RGB")
            image_preproc_dir.mkdir(parents=True, exist_ok=True)
            image.save(image_preproc_path, "JPEG")
        except Exception:
            return None
    return str(PurePath(image_preproc_path))


def preprocess_images(hentai: pd.Series):
    return [
        preprocess_image(filename)
        for filename in hentai["filenames"]
    ]

In [74]:
tqdm.pandas()
hentais_df = hentais_df.assign(
    filenames=hentais_df.progress_apply(preprocess_images, axis=1)
)

100%|██████████| 3341/3341 [01:24<00:00, 39.40it/s] 


In [75]:
hentais_df = hentais_df[hentais_df["filenames"].apply(all)]
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages,filenames
0,355066,BETWEEN US Ch. 1-36,,https://nhentai.net/g/355066,https://nhentai.net/api/gallery/355066,1888032,1618291299,1796,"[big breasts, full censorship, story arc, mast...",[],[],[],"[english, translated]",[],[manga],https://t.nhentai.net/galleries/1888032/cover.jpg,https://t.nhentai.net/galleries/1888032/thumb.jpg,[https://i.nhentai.net/galleries/1888032/1.jpg...,287,"[data\preprocessed\355066\1.jpg, data\preproce..."
1,338065,NI+CORA de SHI+CORA Night,,https://nhentai.net/g/338065,https://nhentai.net/api/gallery/338065,1788827,1606885806,314,"[sole male, dark skin, ffm threesome, group, a...",[charan poran],[tokyo 7th sisters],"[alessandra susu, musubi tendouji]",[japanese],[nekono matatabi],[doujinshi],https://t.nhentai.net/galleries/1788827/cover.jpg,https://t.nhentai.net/galleries/1788827/thumb.jpg,[https://i.nhentai.net/galleries/1788827/1.jpg...,14,"[data\preprocessed\338065\1.jpg, data\preproce..."
2,352167,Boku wa Chiisana Succubus no Shimobe - I am a ...,None,https://nhentai.net/g/352167,https://nhentai.net/api/gallery/352167,1872914,1616460447,20119,"[wings, garter belt, group, uncensored, story ...",[],[],[],"[english, translated]",[tamano kedama],[manga],https://t.nhentai.net/galleries/1872914/cover.jpg,https://t.nhentai.net/galleries/1872914/thumb.jpg,[https://i.nhentai.net/galleries/1872914/1.jpg...,195,"[data\preprocessed\352167\1.jpg, data\preproce..."
3,212548,Syoku,None,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],[occultic nine],[ririka nishizono],"[english, translated]",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33,"[data\preprocessed\212548\1.jpg, data\preproce..."
4,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,None,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],[],[],[japanese],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166,"[data\preprocessed\57170\1.jpg, data\preproces..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3336,19145,Fuuka Meiga za Nihon date: Roommate no Hime go...,None,https://nhentai.net/g/19145,https://nhentai.net/api/gallery/19145,68021,1403984982,17,"[lolicon, yuri]",[bolze.],[mai-otome],"[shizuru fujino, erstin ho, nina wang, yohko h...",[japanese],[rit.],[doujinshi],https://t.nhentai.net/galleries/68021/cover.jpg,https://t.nhentai.net/galleries/68021/thumb.jpg,"[https://i.nhentai.net/galleries/68021/1.jpg, ...",18,"[data\preprocessed\19145\1.jpg, data\preproces..."
3337,243825,Count2.4 CGF Soushuuhen,,https://nhentai.net/g/243825,https://nhentai.net/api/gallery/243825,1274983,1534955886,1167,"[bondage, anal, gag, nakadashi, stockings]",[count2.4],[the idolmaster],"[koume shirasaka, mayu sakuma, mika jougasaki,...",[japanese],[nishi],[doujinshi],https://t.nhentai.net/galleries/1274983/cover.jpg,https://t.nhentai.net/galleries/1274983/thumb.jpg,[https://i.nhentai.net/galleries/1274983/1.jpg...,140,"[data\preprocessed\243825\1.jpg, data\preproce..."
3338,230693,ふたりきり,,https://nhentai.net/g/230693,https://nhentai.net/api/gallery/230693,1213137,1524110819,70,"[anal, dark skin, males only, shotacon, yaoi, ...",[],[dragon ball z],"[son goku, turles]",[japanese],[],[doujinshi],https://t.nhentai.net/galleries/1213137/cover.png,https://t.nhentai.net/galleries/1213137/thumb.png,[https://i.nhentai.net/galleries/1213137/1.png...,15,"[data\preprocessed\230693\1.jpg, data\preproce..."
3339,115897,Otoko no Tatakai 18,,https://nhentai.net/g/115897,ht

In [76]:
# The metadata CSV file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "preprocessed" / "metadata.ndjson"
hentais_df = hentais_df[[
    "id",
    "title",
    "num_favorites",
    "tag",
    "group",
    "parody",
    "character",
    "language",
    "artist",
    "category",
    "num_pages",
    "filenames"
]]
hentais_df.to_json(
    metadata_path,
    orient="records",
    lines=True
)
print("Number of preprocessed images :", len(hentais_df))

Number of preprocessed images : 3335
